In [21]:
from IPython import get_ipython
import pandas as pd
import seaborn as sns
import matplotlib


In [22]:
import csv
import glob
import itertools
import math
import os
import statistics
import time
from collections import OrderedDict
from math import *
from os.path import dirname, join

#ROOT_DIR = dirname(dirname(__file__))

import matplotlib as plt
import numpy as np
import pandas as pd
#needed to model temps through year from monthly averages
from scipy.interpolate import *

#eee
#Setting constants for code with classes

SOLAR_CONSTANT = 1360. #W*m^-2
tau = 0.7 #0.7 clear day
STEFAN_BOLTZMANN = 5.670373*10**(-8) #W*m^-2*K^-4
ALBEDO = 0.4 #ground reflectance (albedo) of dry sandy soil ranges 0.25-0.45
#E_G = 0.9 ##surface emissivity of sandy soil w/<2% organic matter approx 0.88
OMEGA = math.pi/12.
u = 0.1 #windspeed in m/s

cp=29.3 #unit = J/(mol*C) #specific heat of air at constant temp
Tb1=[35]
e=0.01675 #earth essentricity 
A_S = 0.8 #shortwave rad absorbance of animal
A_L = 0.95 #longwave rad absorbance of animal
s = 1.0 #proportion of animal in sun

#scenarios= ['undulatus_utah','occipitalis_ecuador','undulatus_AZ','clarki_AZ','ornatus_AZ','graciosus_kolob','graciosus_mtdiablo','scitulus_NM','mucronatus_usa','grammicus_mexico','grammicus_laguna','grammicus_paredon','maculata_nebraska','undulatus_nebraska','undulatus_newjersey','merriami_usa','boskianus_gabal','boskianus_mallahat','jarrovi_AZ','virgatus_AZ','mamorata_durango','stanburiana_durango','podacris_hispanica_salamanca','psammodromus_algirus_salamanca','psammodromus_hispanicus_salamanca','schreiberi_salamanca','lepida_ciudadrealspain','cantabrica_asturias','vivipara_asturias','cyreni_avila','hispanica_asturias','muralis_asturias','atrata_columbretes','erythrurus_madrid','septentrionalis_xinshau','tachydromoides_honsu','vivipara_antwerpen','agilis_limberg','viridis_loire','draconoides_maricopa','ornatus_maricopa','cornutum_cochise','maculata_cochise','modestum_cochise','ornatus_cochise','texanus_cochise','undulatus_cochise','wislizeni_nevada','undulatus_yavapai','ornatus_yuma','pectinata_morelos','monticola_mandeo','hispanicus_highland_central_mexico','hispanicus_lowland_central_mexico','undulatus_southcarolina','undulatus_texas','undulatus_ohio','undulatus_colorado','undulatus_consobrinus_newmexico','undulatus_tristichus_newmexico','undulatus_kansas','undulatus_garmani_nebraska','undulatus_georgia','nanuzae_brazil','rubrigularis_queensland','przewalskii_Alax_Zuoqi_China','przewalskii_Alax2_China','przewalskii_shandan']
scenarios= ['TC_2021','MC_2021','LR_2021','WC_2021','RP_2021']


class Individual():
    def __init__(self,scenario,latitude,longitude,altitude,mass,length,width,emissivity,tpref_mean):
        'the classes object that holds the relevant morphological, physiological, and geographic information for the animal'
        self.scenario = scenario #to split up different species and pops
        # self.corresponding_data = None 
        # mapping that defines the scenario to the correct CSV file
        self.scenarioLink= { # dictionary holding scenario microclimate data pairs #scenario Link could be any word
            "TC_2021": "microclim_input/hourly_df_TC_2021.csv",
            "MC_2021": "microclim_input/hourly_df_MC_2021.csv",
            "LR_2021": "microclim_input/hourly_df_LR_2021.csv",
            "WC_2021": "microclim_input/hourly_df_WC_2021.csv",
            "RP_2021": "microclim_input/hourly_df_RP_2021.csv"
       
        }

        self.latitude = latitude #decimal degrees
        self.longitude = longitude #decimal degrees
        self.altitude = altitude #meters
        self.MASS = mass #grams
        self.H = length #SVL in m, variable h in original Sears code
        self.D = width #animal diameter in m, variable d in original Sears code
        #self.S = 1.0 + shade #proportion of animal exposed to direct solar radiation (non-shaded)
        #self.A_S = shortwave_absorbance #absorbance of organism to shortwave radiation
        #self.A_L = longwave_absorbance #absorbance of organism to longwave radiation
        self.E_G = emissivity #ground emissivity
        self.E_S = 0.97 #emissivity of organism
        self.tpref_mean = tpref_mean
        #self.Tb_shade = Tb_shade
        #self.Tb_sun = Tb_sun
        #self.Tpref_min = Tpref_min
        #self.Tpref_max = Tpref_max
        #self.active_hours = annual_activity_hours
    
    def read_relative_path(self, filename):
        file_path = join('/Users/laurenneel/Documents/repo_energetics/energetics-model', filename)
        #print(file_path) - include when you want to verify file
        return file_path
    #def read_relative_path(self, filename):
    #    return join(dirname(dirname(__file__)), filename)
    
    def orbit_correction(self,julian): #solar_rad function in sears code
        'orbital correction for day'
        return (1.+2.*e*math.cos(((2*math.pi)/365.)*julian))
    
    def direct_solar_radiation(self,julian): #solar_rad function in original code combines these two into one function
        'required for solar radiation'
        return self.orbit_correction(julian)*SOLAR_CONSTANT
        
    def f(self,julian): #f_var
        'requried for solar declination and equation of time'
        return 279.575 + (0.9856 * julian)
    
    def ET(self,julian):
        'equation of time'
        _f = radians(self.f(julian))
        return (-104.7*math.sin(_f)+596.2*math.sin(2*_f)+4.3*math.sin(3*_f)-12.7*math.sin(4*_f)-429.3*math.cos(_f)-2.0*math.cos(2*_f)+19.3*math.cos(3*_f))/3600.
    
    def LC(self,lon):
        'longitude correction'
        return ((lon%15)*4.0)/60

    def t0(self,lc,et):
        'solar noon'
        t = 12 + lc - et
        return t

    def hour(self,t,t_zero): #hour function in sears code, degrees of rotation needed to rotate the earth to specific number of hours
        'conversion for zenith angle'
        h = 15*(t-t_zero)
        return h

    def declin(self,julian): #declination angle, angle between equator and the suns rays. Depends only on time of year, not time of day.
        'declination angle'
        return degrees(math.asin(0.39785* math.sin(radians(278.97 + 0.9856 * julian + 1.9165 * math.sin(radians(356.6 + 0.9856 * julian))))))
        
    def zenith(self,julian,t): #Lambert's law says intensity of radiation is proportional to the cos of the angle between the suns rays and the normal to the irridiated surface
        'zenith angle'
        if math.acos(math.sin(radians(self.latitude))*math.sin(radians(self.declin(julian))) + math.cos(radians(self.latitude))*math.cos(radians(self.declin(julian)))*math.cos(radians(self.hour(t,(self.t0(self.LC(self.longitude),self.ET(julian))))))) >= 0.:
            return math.acos(math.sin(radians(self.latitude))*math.sin(radians(self.declin(julian))) + math.cos(radians(self.latitude))*math.cos(radians(self.declin(julian)))*math.cos(radians(self.hour(t,(self.t0(self.LC(self.longitude),self.ET(julian)))))))
        return 0.
      
    def m(self,julian,hrs):
        'optical air mass number'
        p_a = 101.3*math.exp(-self.altitude/8200) #p_a = air pressure
        if math.cos(self.zenith(julian,hrs))>=0.:
            return p_a/(101.3*(math.cos(self.zenith(julian,hrs))))
        return 0.
            
    def hS0(self,julian,hrs): #solar rad adjusted by cos of zenith angle
        'direct solar radiation'
        z = self.zenith(julian,hrs)
        if math.cos(z)>= 0.:
            return self.direct_solar_radiation(julian)*(math.cos(z))
        return 0.
            
    def hS(self,julian, hrs, tau): #correcting for radiation hitting the atmosphere to get DIRECT incident solar rad, hS
        'solar radiation, corrected'
        return self.hS0(julian,hrs)*tau**self.m(julian,hrs)

    def diffuse_solar(self,julian,hrs,tau): #Sd, diffuse shortwave, solar rad scattered from reflecting off clouds
        'diffuse solar radiation'
        return self.hS0(julian,hrs)*0.3*(1.-(tau**self.m(julian,hrs)))

    def reflected_radiation(self,julian,t,tau): #Sr, reflected shortwave rad
        'reflected solar radiation'
        return ALBEDO*self.hS(julian,t,tau)
               
    def view_factor(self,julian,t):
        'view factor, Fh'
        return (1.+((4.*(self.H)*math.sin(radians(degrees(self.zenith(julian,t)))))/(math.pi*(self.D))))/(4.+(4.*(self.H)/(self.D)))

    def longwave_sky(self, Ta): #Sla, only varies with Ta
        'longwave radiation from sky'
        return 53.1*10**-14*(Ta +273.15)**6.

    def longwave_ground(self,Tg):
        'longwave radiation from the ground'
        return self.E_G*STEFAN_BOLTZMANN*(Tg +273.15)**4.
      
    def dynamic_frame_load(self):
        # read in the CSV
        # concatenate the directory with the file name using join function
        # Join and adding strings with "+" operator are same
        return pd.read_csv(self.read_relative_path(self.scenarioLink[self.scenario]))
    
    def radiation_abs(self, julian, hour, s, A_S, tau, Ta, Tg, A_L):
        'total absorbed radiation'
        dynamic_data_frame = pd.read_csv(self.read_relative_path(self.scenarioLink[self.scenario]))
        if math.cos(self.zenith(julian,hour)) > 0.:
            return (s*A_S*(self.view_factor(julian,hour) * (self.hS(julian, hour, tau)) + (0.5*self.diffuse_solar(julian, hour, tau)) + (0.5*self.reflected_radiation(julian, hour, tau)))) + (0.5*A_L*(self.longwave_sky(Ta)+ self.longwave_ground(Tg)))
        else:
            return (0.5*A_L*(self.longwave_sky(Ta) + self.longwave_ground(Tg)))

    def gha(self, u): #u = wind speed m/s
        'boundary layer conductance for hear assuming d=diameter, Table 7.5 Campbell and Norman'
        return 1.4*(0.135*math.sqrt(u/self.D))
    
    def gr(self, Ta, cp):
        'radiative conductance of the animal, Ta needs to be at height of animal'
        return ((4*STEFAN_BOLTZMANN)*((Ta +273.15)**3.))/cp
      
    def operative(self, Ta, julian, hour, s, A_S, tau, Tg, A_L, cp, u):
        return Ta + ((self.radiation_abs(julian, hour, s, A_S, tau, Ta, Tg, A_L) - self.E_S*STEFAN_BOLTZMANN*Ta**4) / (cp*(self.gr(Ta, cp)+self.gha(u))))
    
    def Tb2(self, Te, Tb1, time_at_temp, time_constant):
        return Te + ((Tb1-Te)*(math.exp(-time_at_temp/time_constant)))

    # def activity_status_5C(self, Te, Tb1, time_at_temp, time_constant):
    #     activity_status = [1 if (Tpref_min <= shade <= Tpref_max or Tpref_min <= sun <= Tpref_max) else 0 for shade, sun in zip(tb_shade, tb_sun)]


windspeeds = [0.1]# [0.1,1.0,2.0,3.0]
time_at_temp=5.

#species = pd.read_csv(join(ROOT_DIR, 'parameters/jarrovi_input.csv')) 
species = pd.read_csv(join('/Users/laurenneel/Documents/repo_energetics/energetics-model/parameters/jarrovi_input.csv')) 

# hourly_results = pd.DataFrame(columns = ['species','scenario_group','julian','hour','tpref_mean','Rabs_sun','Rabs_shade','Te_sun','Te_shade','Tb_sun','Tb_shade','activity_status_5C','activity_status_25C','activity_status_skewed_5C','activity_status_skewed_10C', 'ro','rcm','growth_k','growth_linf'])
activity_data = []
# hourly_results = pd.DataFrame(columns = ['species','scenario_group','julian','hour','Rabs_sun','Rabs_shade','Te_sun','Te_shade','Tb_sun', 'Tb_shade'])




In [23]:

try:
    for i in range(len(species)):
        failedspecifes = i
        try:
            ectotherm = Individual(scenarios[i],species.latitude[i],species.longitude[i],species.altitude[i],species.mass[i],species.length[i],species.width[i],species.emissivity[i],species.tpref_mean[i])
            #ectotherm = Individual(species.type[i],species.spp[i],species.lizard_location[i], scenarios[i],species.latitude[i],species.longitude[i],species.altitude[i],species.mass[i],species.length[i],species.width[i],species.emissivity[i],species.tpref_mean[i])
        except Exception as e:
            print(e)

        loaded_frame = ectotherm.dynamic_frame_load()

        previous_tb_timestep_sun = 5 #replacing Tb1 dummy start value
        previous_tb_timestep_shade = 5

        for index, row in loaded_frame.iterrows():
            if (index == 2): #12Jan2020 , include when wanting to check if code works but not generate full dataset
                break #12Jan2020 , include when wanting to check if code works but not generate full dataset
            failedhour=index
            julian = row['julian']
            hour = row['hour']
            Ta_sun = row['Ta_sun']
            Ta_shade = row['Ta_shade']
            sun_D0cm = row['Tg_avg']
            shade_D0cm = row['Tg_avg']
            Rabs_sun = ectotherm.radiation_abs(julian, hour, 1., A_S, tau, Ta_sun, sun_D0cm, A_L)
            Rabs_shade = ectotherm.radiation_abs(julian, hour, 0., A_S, tau, Ta_shade, shade_D0cm, A_L)

            Te_sun=ectotherm.operative(Ta_sun, julian, hour, s, A_S, tau, sun_D0cm, A_L, cp, u)
            Te_shade=ectotherm.operative(Ta_shade, julian, hour, s, A_S, tau, shade_D0cm, A_L, cp, u)

            if Te_sun >= previous_tb_timestep_sun:
                time_constant=math.exp(0.72+0.36*log(ectotherm.MASS))
            elif Te_shade >= previous_tb_timestep_shade:
                time_constant=math.exp(0.72+0.36*log(ectotherm.MASS))
            elif Te_sun <= previous_tb_timestep_sun:
                time_constant=math.exp(0.42+0.44*log(ectotherm.MASS))
            elif Te_shade <= previous_tb_timestep_shade:
                time_constant=math.exp(0.42+0.44*log(ectotherm.MASS))
            else:
                print('warning: time_constant shit is fucked')

            Tb_sun= ectotherm.Tb2(Te_sun, previous_tb_timestep_sun, time_at_temp, time_constant)
            Tb_shade= ectotherm.Tb2(Te_shade, previous_tb_timestep_shade, time_at_temp, time_constant)
            previous_tb_timestep_sun = Tb_sun
            previous_tb_timestep_shade = Tb_shade

            activity_status_5C = 0. if Tb_sun > (ectotherm.tpref_mean+5.0) or Tb_shade < (ectotherm.tpref_mean-5.0) else 1.
            activity_status_25C = 0. if Tb_sun > (ectotherm.tpref_mean+2.5) or Tb_shade < (ectotherm.tpref_mean-2.5) else 1.
            # activity_status_skewed_5C = [0. if Tb_sun > (ectotherm.tpref_mean+1.25) or Tb_shade < (ectotherm.tpref_mean-3.75) else 1.]
            # activity_status_skewed_10C = [0. if Tb_sun > (ectotherm.tpref_mean+2.5) or Tb_shade < (ectotherm.tpref_mean-7.5) else 1.]

            # activity_status_5C = [1 if ((Tpref_mean-2.5) <= Tb_shade <= (Tpref_mean+2.5) or (Tpref_mean-2.5) <= Tb_sun <= (Tpref_mean+2.5) else 0 for shade, sun in zip(tb_shade, tb_sun)]

            activity_data.append([scenarios[i], julian, hour, species.tpref_mean[i], Rabs_sun, Rabs_shade, Te_sun, Te_shade, Tb_sun, Tb_shade, activity_status_5C, activity_status_25C])
            # if (index == 2): #comment this line out when generating full results
            #     break #comment this line out when generating full results

    dataframe = pd.DataFrame(activity_data, columns = ['scenario','julian','hour','Tpref_mean','Rabs_sun','Rabs_shade','Te_sun','Te_shade','Tb_sun','Tb_shade','activity_status_5C','activity_status_25C'])
    with open(('/Users/laurenneel/Documents/repo_energetics/energetics-model/jarrovi_output.csv'), 'w') as f:
        dataframe.to_csv(f, header=True)
except Exception as e:
    print(e)




#   summarize results
hourly= pd.read_csv('/Users/laurenneel/Documents/repo_energetics/energetics-model/jarrovi_output.csv')
daily_results = hourly.groupby(['scenario','julian','hour'],as_index=False)
#daily_df = pd.DataFrame([OrderedDict([
#                 ('activity_status_5C' , hourly.groupby(by=["scenario"])["activity_status_5C"].sum().reset_index(name='lauren')),
#                 ('activity_status_25C', hourly.groupby(by=["scenario"])["activity_status_25C"].sum().reset_index(name='jake')),
#                #  ('Te_sun', statistics.mean),
#                #  ('Te_shade' , statistics.mean),
#                #  ('Tb_sun' , statistics.mean),
#                #  ('Tb_shade',statistics.mean),
#                ])], columns=["sum_activity_status_5C", "sum_activity_status_25C"])
##daily_df = pd.merge(hourly.groupby(by=["scenario"])["activity_status_5C"].sum().reset_index(name='sum_activity_status_5C'), hourly.groupby(by=["scenario"])["activity_status_25C"].sum().reset_index(name='sum_activity_status_25C'), how='inner', on=['scenario'])
daily_results.to_csv(('/Users/laurenneel/Documents/repo_energetics/energetics-model/daily_output.csv'),index = False)              
resultant_df = pd.merge(hourly.groupby(by=["scenario"])["activity_status_5C"].sum().reset_index(name='sum_activity_status_5C'), hourly.groupby(by=["scenario"])["activity_status_25C"].sum().reset_index(name='sum_activity_status_25C'), how='inner', on=['scenario'])
resultant_df.to_csv(('/Users/laurenneel/Documents/repo_energetics/energetics-model/jarrovi_output.csv'),index = False)


AttributeError: 'DataFrameGroupBy' object has no attribute 'to_csv'

In [20]:
#display(dataframe)
daily_results.to_frame()

daily_results.to_csv('daily.csv')
#plt.plot(dataframe['hour'], dataframe['Tb_sun'])

#ax=sns.regplot(x='hour',y='Tb_sun', data=dataframe, color='green')


AttributeError: 'DataFrameGroupBy' object has no attribute 'to_frame'

In [2]:

################################ set constants  ######################

#earth essentricity 
e=0.01675 

#So = solar constant = 1360 W m-2
S0=1360 #Wm-2

#boltzman constant, sigma = 5.67e-8
sigma=5.67e-8 # W m-2 k-4

#ordinal day of the year
julian_days= np.arange(1,366,1)

# eg = ground emissivity
eg=0.75


time_list_day=np.arange(0,24,0.1)
#julian=4
latitude=31.85088889
longitude=109.3256389
altitude=2015
tau=0.7
rg=0.4 #ground reflectance #when modeled with sears April2019 we used rg=0.4
h=0.1 #SVL of 100mm
d=0.1 #animal diameter in unit meters 

Tave=25.
amp=15.
z=0.
D=0.03
u=0.1 #u=wind in m/s

#max_t_yesterday=32.2
#min_t_today=17.6
#max_t_today=32.2
#min_t_tomorrow=17.6


# alpha_s = shortwave rad absorptance of the animal 
alpha_s = 0.8

# alpha_l = longwave rad absorptance of the animal
alpha_l=0.95

#cp = specific heat of air at constant temp 
cp=29.3 #unit = J/(mol*C)

mass=20


##### constants that differ by location ####
tc_latitude = 31.85088889
tc_longitude = 109.3256389
tc_altitude = 1998.


In [ ]:
# ############################### define functions  #####################

# #dd2 = d over d sqared.. correction factor for eliptical orbit of earth at a given J, julian calendar day
# #hSo = amount of direct solar radiation reaching earths surface = So*(dd2**2)

# def solar_rad(julian):
#     return S0*(1.+2.*e*math.cos(((2*math.pi)/365.)*julian))
    
# def f_var(J):
#     return 279.575 + (0.9856*J)

# #ET = equation of time    
# def ET(J):
#     f=f_var(J) ### then you wouldn't need to call the function each time
#     return (((-104.7*math.sin(radians(f)))+(596.2*math.sin(radians(2*f)))+(4.3*math.sin(radians(3*f)))-(12.7*math.sin(radians(4*f)))-(429.3*math.cos(radians(f)))-(2*math.cos(radians(2*f)))+(19.3*math.cos(radians(3*f))))/3600)

# #LC = longitudinal correction
# def LC(long):
#     return (long % 15.)/15.

# #t0 = solar noon
# def t0(J,long):
#     return 12. - ET(J) + LC(long)

# #h = hour angle, the degrees of rotation needed to rotate the earth to a specific number of hours
# def hour(time,J,long):
#     return 15.*(time-t0(J,long))

# #δ = declination angle, angle between the equator and the sun'r rays. Depends only on time of year, not time of day.
# def declination(J):  
#     #solar declination ranges from +23.45 degrees at summer solstice to -23.45 at winter solstics
#     return degrees(math.asin(0.39785*math.sin(radians(278.87+0.9856*J+1.9165*math.sin(radians(356.6+0.9856*J))))))

# z = zenith angle
# def zenith(lat,J,time,long):     #Lambert's cos law says the intensity of radiation is proportional to the cos of the angle between the suns rays and the normal to the irridiated surface
#     #declin=declination(J)
#     #h=hour(time,long,J)
#     #return math.acos(math.sin(radians(lat))*math.sin(radians(declin))+math.cos(radians(lat))*math.cos(radians(declin))*math.cos(radians(h)))
#     return math.acos(math.sin(radians(lat))*math.sin(radians(declination(J)))+ math.cos(radians(lat))*math.cos(radians(declination(J)))*math.cos(radians(hour(time,J,long))))

# #solar rad adjusted by the cosine of zenith angle
# def hS0(J,lat,time,long):
#     return solar_rad(J) * math.cos(zenith(lat,J,time,long))

# #Up to now we have direct radiation hitting atmosphere.. Beer's law accounts for the atmosphere. Longer distance through the atm to the ground, less radiation. Why radiation is more intense at high elevation

# #p_a = the air pressure
# def pa(alt): #altitude (a) is in meters
#     return 101.3*math.exp((-alt)/8200)

# m = optical air mass number
# def air_mass(alt,lat,time,long,J):
#     z=zenith(lat,J,time,long)
#     p_a=pa(alt)
#     return p_a/(101.3*math.cos(z))

# #correcting for radiation hitting the atmosphere to get DIRECT incident solar radiation, hS
def hS(J,lat,time,long,tau,alt):  #Tau = optical transmittance of the atmosphere. 
    #T varies as function of the clarity of the atmosphere. T is 0.75 on a clear day. More typically between 0.6-0.7 on clear day. 
    #Values < 0.4 represent cloudy days
    m=air_mass(alt,lat,time,long,J)
    return solar_rad(J)*(math.cos(zenith(lat,J,time,long)))*(tau**m)


Sd = diffuse shortwave radiation... solar rad that's scatted from reflecting off clouds, particulate matter, atc. 
def Sd(J,lat,time,long,tau,alt):
    return hS0(J,lat,time,long) * 0.3*(1-tau**(degrees(air_mass(alt,lat,time,long,J))))



## Won't need any of the functions above because we're getting shortwave radiation (direct and diffuse) at each time from GRASS Input table



In [ ]:
############################### define functions  #####################


## Won't need any of the functions above because we're getting shortwave radiation (direct and diffuse) at each time from GRASS Input table


# Sr = reflected shortwave radiation
def Sr(rg,J,lat,time,long,tau,alt):
    return rg*(hS(J,lat,time,long,tau,alt)+ Sd(J,lat,time,long,tau,alt))

#Thermal radiation (longwave)
# Sla = longwave radiation from the atmosphere, Sla only varies with air temp 
def Sla(Ta):
    return 53.1*(10**-14)*(Ta+273.15)**6

# Slg = longwave radiation emitted from ground ##OUT
def Slg(eg,Tg):  #ground emissivity
    return eg*sigma*(Tg+273.15)**4

# Longwave radiation heating ground from above (going IN), uses sky view input from GRASS
def longwave_rad_IN(Sview, Ta,thermal_rad_veg_above):
    return (Sview * Sla(Ta))+ ((1-Sview)* thermal_rad_veg_above)

# Net Longwave Radiation = IN - OUT 
def Net_longwave(Sview, Ta,thermal_rad_veg_above,eg,Tg):
    longwave_IN = longwave_rad_IN(Sview, Ta,thermal_rad_veg_above)
    longwave_OUT = Slg(eg,Tg)
    return longwave_IN - longwave_OUT

#Gamma_t = dimensionless diurnal temp function
#def Gamma_t(t):
#    return 0.44-0.46*math.sin((math.pi/12)*t+0.9) + 0.11*math.sin(2*(math.pi/12)*t + 0.9)
#
#def anytime_temp(t,max_t_yesterday,min_t_today,max_t_today,min_t_tomorrow):
#    #t = time
#    #i = today
#    #i-1 = yesterday
#    #i+1 = tomorrow
#    #Tn = daily min
#    #Tx = daily max
#    if 0.<=t<=5.:
#        return max_t_yesterday*Gamma_t(t) + min_t_today*(1-Gamma_t(t))
#    elif 5.<t<=14.:
#        return max_t_today*Gamma_t(t)+ min_t_today*(1-Gamma_t(t))
#    else:
#        return max_t_today*Gamma_t(t)+ min_t_tomorrow*(1-Gamma_t(t))
    
#statistically predicting Tg at a given depth (z) and time of day (t)
##Tg = ground temperatures
#def Tzt(Tave,amp,z,D,t):
#    #Tave = average daily soil temp
#    #A(0) = amplitude (half of the peak to peak variation) of temp fluctuations at the suface
#    #D = damping depth (D=0.1 for moist mineral soils and 0.03-006 for dry mineral soils)
#    return Tave+amp*math.exp(-z/D)*math.sin((math.pi/12)*(t-8)-z/D)

# Fh = view factor .....## This is same as sky view factor, correct???
def Fh(h,lat,J,time,long,d):
    zenith_angle=zenith(lat,J,time,long)
    return(1+((4*h*math.sin(zenith_angle))/(math.pi*d)))/(4+((4*h)/d))

# Rabs = amount of radiation absorbed by an organism    
# def Rabs(s,alpha_s,h,lat,J,time,long,d,tau,alt,rg,alpha_l,max_t_yesterday,min_t_today,max_t_today,min_t_tomorrow,eg,Tave,amp,z,D):
#     if math.cos(zenith(latitude,julian,time,longitude)) > 0.:
#         return s*alpha_s*(Fh(h,lat,J,time,long,d)*(hS(J,lat,time,long,tau,alt))+0.5*Sd(J,lat,time,long,tau,alt)+0.5*Sr(rg,J,lat,time,long,tau,alt)) + 0.5*alpha_l*(Sla(anytime_temp(time,max_t_yesterday,min_t_today,max_t_today,min_t_tomorrow)) + Slg(eg,Tzt(Tave,amp,z,D,time))) 
#     else:
#         return 0.5*alpha_l*(Sla(anytime_temp(time,max_t_yesterday,min_t_today,max_t_today,min_t_tomorrow)) + Slg(eg,Tzt(Tave,amp,z,D,time))) 


# function below was modified to calculate Rabs for 1 temp input        
# def Rabs(s,alpha_s,h,lat,J,time,long,d,tau,alt,rg,alpha_l,Ta,eg,Tave,amp,z,D):
#     if math.cos(zenith(latitude,julian,time,longitude)) > 0.:
#         return s*alpha_s*(Fh(h,lat,J,time,long,d)*(hS(J,lat,time,long,tau,alt))+0.5*Sd(J,lat,time,long,tau,alt)+0.5*Sr(rg,J,lat,time,long,tau,alt)) + 0.5*alpha_l*(Sla(anytime_temp(time,max_t_yesterday,min_t_today,max_t_today,min_t_tomorrow)) + Slg(eg,Tzt(Tave,amp,z,D,time))) 
#     else:
#         return 0.5*alpha_l*(Sla(Ta)) + Slg(eg,Tzt(Tave,amp,z,D,time))     



### New Rabs function to incoporate GRASS input table
def Rabs()

    
# gha = boundary layer conductance for heat assuming d = diameter see Table 7.5 campbell and norman
def gha(u,d):
    #u = wind speed (unit m/s)    
    return 1.4*(0.135*math.sqrt(u/d))

# gr = radiative conductance of the animal
def gr(sigma,Ta,cp): #Ta needs to be air temp at height of animal
    return ((4*sigma)*((Ta+273.15)**3))/cp

def operative(Ta,Rabs,eg,sigma,cp,u,d):
    return Ta + ((Rabs-eg*sigma*Ta**4)/cp*(gr(sigma,Ta,cp)+gha(u,d)))

#calculating body temperatures p 202
Tb1=[25]
def Tb2(Te,Tb1,time_at_temp,time_constant):
    return Te + ((Tb1-Te)*(math.exp(-time_at_temp/time_constant)))



                         

In [ ]:
time_list = np.arange(0, 24, 0.1).tolist()
print(type(time_list))

new_time = time_list * 37




Sla_list =[]
for val in Ta_shade:
    Sla_val = Sla(val)
    Sla_list.append(Sla_val)
#plot(Ta_shade,Sla_list)
#ylabel('longwave radiation')
#xlabel('downscaled Ta from NicheMapR')


#what am I trying to do?
### print Rabs list as long as Ta_shade list

#how long is Ta_shade list?
### hourly Ta ... 365 days * 24 hours in day


Rabs_vals_sun1=[]
Rabs_vals_shade1=[]
for val,time,julian in zip(Ta_shade,new_time,DOY):
    Sla_val= Sla(val)
    Slg_val= Slg(eg,Tzt(Tave,amp,z,D,time))
    if math.cos(zenith(latitude,julian,time,longitude)) > 0.:
        S_direct= hS(julian,latitude,time,longitude,tau,altitude)
        S_diffuse= Sd(julian,latitude,time,longitude,tau,altitude)
        S_reflected= Sr(rg,julian,latitude,time,longitude,tau,altitude)
        s=1. #full sun
    else:
        S_direct=0.
        S_diffuse=0.
        S_reflected=0.
        s=0. #full shade
    Rabs_vals_sun1.append(Rabs(s,alpha_s,h,latitude,julian,time,longitude,d,tau,altitude,rg,alpha_l,val,eg,Tave,amp,z,D))   
    Rabs_vals_shade1.append(Rabs(0.,alpha_s,h,latitude,julian,time,longitude,d,tau,altitude,rg,alpha_l,val,eg,Tave,amp,z,D))   
        
DOY_cut,Rabs_cut=[],[]
        
for j,r in zip(DOY,Rabs_vals_sun1):
    if 40<j<60:
        Rabs_cut.append(r)
        DOY_cut.append(j)
        
plot(DOY_cut,Rabs_cut)

#plot(DOY,Rabs_vals_sun1)
title('annual absorbed radiation for animal in full sun - Tempe 2019 ')
ylabel('Rabs (W/m2)')
xlabel('day of year')
#xlim(1,365)
     
    
    
#240
#8760




In [ ]:

operative_sun1,operative_shade1=[],[]
for julian,time,val in zip(DOY,new_time,Ta_shade):
    Rabs_sun= Rabs(1.,alpha_s,h,latitude,julian,time,longitude,d,tau,altitude,rg,alpha_l,val,eg,Tave,amp,z,D)
    operative_sun1.append(operative(val,Rabs_sun,eg,sigma,cp,u,d))
    Rabs_shade=Rabs(0.,alpha_s,h,latitude,julian,time,longitude,d,tau,altitude,rg,alpha_l,val,eg,Tave,amp,z,D)
    operative_shade1.append(operative(val,Rabs_shade,eg,sigma,cp,u,d))
    
plot(dates,operative_sun1,label='Te full sun')
plot(dates,operative_shade1,label='Te shade')
legend()
#title('Operative temperatures for 100mm SVL lizard in Tempe, AZ')
ylabel('Temperature C')
xlabel('day of year')
#ylim(20,45)


# for j,r in zip(DOY,Rabs_vals_sun1):
#     if 40<j<60:
#         Rabs_cut.append(r)
#         DOY_cut.append(j)



In [ ]:


from collections import defaultdict

daily_temps = defaultdict(list)

for day, value in zip(DOY,Ta_sun):
    daily_temps[day].append(value)
    
ranges = dict()
for day, value in daily_temps.items():
    ranges[day] = (min(value), max(value))

    



